In [ ]:
#Q1 Solution
import requests
import pandas as pd

class StockData:
    API_KEY = "ZP73Q3Y1268DQA3T"
    BASE_URL = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=1min&apikey={}"

    def __init__(self):
        self.intraday_data = {}
    
    def fetch_intraday_data(self, script):
        url = self.BASE_URL.format(script, self.API_KEY)
        response = requests.get(url)
        self.intraday_data[script] = response.json()
    
    def convert_intraday_data(self, script):
        data = self.intraday_data[script]
        time_series = data["Time Series (1min)"]
        df = pd.DataFrame.from_dict({timestamp: {
            "open": float(row["1. open"]),
            "high": float(row["2. high"]),
            "low": float(row["3. low"]),
            "close": float(row["4. close"]),
            "volume": int(row["5. volume"])
        } for timestamp, row in time_series.items()}, orient="index")
        df.index = pd.to_datetime(df.index)
        self.intraday_data[script] = df
    
    def __getitem__(self, script):
        return self.intraday_data[script]
    
    def __setitem__(self, script, data):
        self.intraday_data[script] = data
    
    def __contains__(self, script):
        return script in self.intraday_data


In [ ]:
# Q2 Solution
def indicator1(df, timeperiod):
    return pd.DataFrame({
        "timestamp": df.index,
        "indicator": df["close"].rolling(timeperiod).mean()
    })


In [ ]:
# Q3 Solution
class Strategy:
    def __init__(self, script_data, script_name):
        self.script_data = script_data
        self.script_name = script_name

    def fetch_intraday_data(self):
        self.df = self.script_data.fetch_intraday_data(self.script_name)
        self.close_data = self.df["close"]

    def compute_indicator_data(self, timeperiod):
        self.indicator_data = indicator1(self.df, timeperiod)["indicator"]

    def generate_signals(self):
        signals = []
        for i in range(1, len(self.df)):
            if self.indicator_data[i] > self.close_data[i] and self.indicator_data[i - 1] <= self.close_data[i - 1]:
                signals.append("BUY")
            elif self.indicator_data[i] < self.close_data[i] and self.indicator_data[i - 1] >= self.close_data[i - 1]:
                signals.append("SELL")
            else:
                signals.append("NO_SIGNAL")
        self.signals = pd.DataFrame({
            "timestamp": self.df.index[1:],
            "signal": signals
        })

    def print_signals(self):
        print(self.signals[self.signals["signal"].isin(["BUY", "SELL"])])
